![Alt text: Slogan da Meteora.](https://i.imgur.com/VDYh2G1.png)

# 1. Estruturando a tabela

**Meteora** é uma loja que vende roupas e acessórios de diversas marcas por todos os estados do Brasil, para entender suas bases de dados e exibir informações relevantes com o objetivo de auxiliar suas tomadas de decisão.

In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect, text
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


**Sobre as tabelas**

* itens_pedidos: tabela que informa sobre o que foi vendido, vai ter informação o preço, quantidade, para onde vai ser enviado e o frete do pedido
* pedidos: vai falar sobre a venda feita, tendo informação sobre os vendedores, preço do pedido e a data da venda
* produtos: informa características dos produtos que são comercializados na loja
* vendedores: informa o nome dos vendedores.

In [2]:
# urls com as fonte dos dados que serão utilizadas nesse novo estudo

url_itens_pedidos = 'https://github.com/alura-cursos/SQL-python-integracao/raw/main/TABELAS/itens_pedidos.csv'
url_pedidos = 'https://github.com/alura-cursos/SQL-python-integracao/raw/main/TABELAS/pedidos.csv'
url_produto = 'https://github.com/alura-cursos/SQL-python-integracao/raw/main/TABELAS/produtos.csv'
url_vendedores = 'https://github.com/alura-cursos/SQL-python-integracao/raw/main/TABELAS/vendedores.csv'

In [3]:
itens_pedidos = pd.read_csv(url_itens_pedidos)
pedidos = pd.read_csv(url_pedidos)
produtos = pd.read_csv(url_produto)
vendedores = pd.read_csv(url_vendedores)

Utilzaremos nessa estudo exploratorio o SQLite para criação dos nosso banco de dados local, assim teremos um estudo mais dinâmico e eficiente para esse caso, do que teriamos utilizando outro SGBDs.

In [4]:
# criação da engine responsavel pela criação das tabelas com sqlite

engine = create_engine('sqlite:///:memory:')

In [5]:
pedidos.to_sql('pedidos', engine, index=False)
itens_pedidos.to_sql('itens_pedidos', engine, index=False)
produtos.to_sql('produtos', engine, index=False)
vendedores.to_sql('vendedores', engine, index=False)

5

In [6]:
# inspecionando tabelas criadas

inspector = inspect(engine)
inspector.get_table_names()

['itens_pedidos', 'pedidos', 'produtos', 'vendedores']

# 2. Primeiras consultas

Vamos entender a característica do produto que é comercializadora na Meteora, analisando a **condição** dos produtos vendidos.

In [7]:
# definindo uma função para consulta em nosso banco de dados

def consulta(query):
  with engine.connect() as conexao:
    pesquisa = conexao.execute(text(query))
    dados = pesquisa.fetchall()
  return pd.DataFrame(dados, columns=pesquisa.keys())

In [8]:
# testando consulta

query = ''' SELECT CONDICAO FROM PRODUTOS '''

consulta(query)

,Condicao
0,Usado
1,Usado
2,Usado
3,Usado
4,Usado
...,...
200,Usado
201,Usado
202,Usado
203,Usado


Buscaremos entender como estão dispersos as condições de nossos produtos, quais são as condições existentes? E quantos produtos existem por cada condição?

In [9]:
# realizando consulta de condições

query = ''' SELECT CONDICAO, COUNT(*) AS Quantidade
FROM PRODUTOS
GROUP BY CONDICAO '''

dados_condicao = consulta(query)
dados_condicao

,Condicao,Quantidade
0,Novo com etiqueta,22
1,Novo sem etiqueta,7
2,Usado,176


In [10]:
# criando visualização de grafico das condições dos produtos

fig = px.bar(dados_condicao, x='Condicao', y='Quantidade', color='Condicao', title='Quantidade de Produtos por Condição')
fig.update_layout(xaxis_title='', yaxis_title='')
fig.show()

Agora precisamos ranquear os produtos que mais foram pedidos por **quantidade** para entender a necessidade de um estoque de produtos na loja.

In [11]:
# visualizando tabelas

consulta('SELECT * FROM ITENS_PEDIDOS').head(5)

,id_nf,produto_id,pedido_id,quantidade,valor_unitario,valor_total,Estado,frete
0,1,41518,341,3,260,780,BR-BA,156.0
1,2,4307,1174,5,6175,30875,BR-RJ,6175.0
2,3,22407,1399,3,200,600,BR-PB,120.0
3,4,3820,1652,6,139,834,BR-DF,166.8
4,5,29012,2470,3,525,1575,BR-BA,315.0


In [12]:
consulta('SELECT * FROM PRODUTOS').head(5)

,produto_id,produto,preco,marca,sku,Condicao
0,21244,Oculos Lente Azulada,1120,D&g Dolce & Gabbana,209297,Usado
1,9981,Bolsa Coral Saco,4000,Givenchy,278612,Usado
2,84176,Camisa Xadrez Verde,310,Joe Fresh,322482,Usado
3,47475,Calca Alfaiataria Preta,490,Mixed,263658,Usado
4,74864,Vestido Jeans Babados,130,Zara,219248,Usado


In [13]:
# criando query para puxar quantidade vendida por produto

query = '''SELECT ITENS_PEDIDOS.PRODUTO_ID, PRODUTOS.PRODUTO, SUM(ITENS_PEDIDOS.QUANTIDADE) AS quantidade
FROM ITENS_PEDIDOS, PRODUTOS
WHERE ITENS_PEDIDOS.PRODUTO_ID = PRODUTOS.PRODUTO_ID
GROUP BY PRODUTOS.PRODUTO
ORDER BY QUANTIDADE DESC'''

dados_quantidade = consulta(query)
dados_quantidade

,produto_id,produto,quantidade
0,15875,Vestido Recortes Cores,795
1,98640,Calca Jeans Costuras,755
2,72667,Saia Pregas Preta,751
3,11218,Calca Alfaiataria Preta,750
4,32126,Saia Midi Cinto,549
...,...,...,...
196,2706,Blusa Cinza,341
197,32986,Blusa Babados Off-white,337
198,88368,Blusa Malha Azul,325
199,84743,Blusa Listrada Malha,322


In [14]:
# construindo grafico dos 10 produtos mais vendidos

fig = px.bar(dados_quantidade.head(10), x='quantidade', y='produto', color='produto', title='10 Produtos Mais Vendidos')
fig.update_layout(xaxis_title='Quantidade Vendida', yaxis_title='')
fig.show()

Continua nossa analise, veremos também quais produtos tivrem o maior volume de receita.

In [15]:
# realizando query para puxarmos os dados de produtos com maiores receitas

query = ''' SELECT ITENS_PEDIDOS.PRODUTO_ID, PRODUTOS.PRODUTO, SUM(ITENS_PEDIDOS.VALOR_TOTAL) AS receita
FROM ITENS_PEDIDOS, PRODUTOS
WHERE ITENS_PEDIDOS.PRODUTO_ID = PRODUTOS.PRODUTO_ID
GROUP BY PRODUTOS.PRODUTO
ORDER BY RECEITA DESC '''

dados_receita = consulta(query)
dados_receita

,produto_id,produto,receita
0,3865,Bolsa Classica Roxa,2860000
1,4307,Bolsa Intrecciato Caramelo,2216825
2,100932,Sapato Cetim Pink,1821300
3,89100,Vestido Longo Preto,1814400
4,17990,Saia Couro Tijolo,1751800
...,...,...,...
196,73692,Shorts Estampa Floral,35640
197,75706,Shorts Poa Pb,35600
198,38723,Top Cropped Coqueiros,31440
199,86544,Macaquinho Tricot Bege,20650


In [16]:
# criando visualização de produtos com maiores receitas

fig = px.bar(dados_receita.head(10), x='receita', y='produto', color='produto', title='10 Produtos com Maior Receita')
fig.update_layout(xaxis_title='Receita', yaxis_title='')
fig.show()

Queremos agora descobrir quais foram as marcas mais pedidas em quatidade de vendas, isso pode nós ajudar a entender se temos alguma marca mais popular ou menos polular

In [17]:
# criando query para buscarmos as mais vendidas

query = ''' SELECT ITENS_PEDIDOS.PRODUTO_ID, PRODUTOS.MARCA, SUM(ITENS_PEDIDOS.QUANTIDADE) AS quantidade
FROM ITENS_PEDIDOS, PRODUTOS
WHERE ITENS_PEDIDOS.PRODUTO_ID = PRODUTOS.PRODUTO_ID
GROUP BY PRODUTOS.MARCA
ORDER BY QUANTIDADE DESC '''

dados_marca = consulta(query)
dados_marca

,produto_id,marca,quantidade
0,72621,Zara,9545
1,31835,Mixed,5748
2,61071,Animale,4013
3,45695,Le Lis Blanc,2479
4,41518,Banana Republic,2045
...,...,...,...
112,66255,All Saints Spitalfields,349
113,40508,Express,342
114,18790,Bcbgmaxzria,333
115,88368,Talie Nk,325


In [18]:
# criando visualização das marcas mais vendidas

fig = px.bar(dados_marca.head(5), x='marca', y='quantidade', color='marca', title='Marcas Mais Vendidas')
fig.update_layout(xaxis_title='', yaxis_title='Quantidade')
fig.show()

# 3. Lidando com filtro

Vamos iniciar uma análise de desempenho do **time de vendedores** da Meteora. A intenção aqui é entender como se deram as vendas dos vendedores no **ano anterior** para que possam ser aplicadas promoções e bônus salarial para a equipe no ano atual da tabela que é 2021

In [19]:
# vamos começar verificando a tabela pedidos

consulta('SELECT * FROM PEDIDOS').head(5)

,pedido_id,produto_id,vendedor_id,data_compra,total
0,341,41518,5,2019-01-05,780
1,1174,4307,3,2019-01-05,30875
2,1399,22407,1,2019-01-05,600
3,1652,3820,4,2019-01-05,834
4,2470,29012,2,2019-01-05,1575


In [20]:
consulta('SELECT * FROM PEDIDOS').info(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24527 entries, 0 to 24526
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   pedido_id    24527 non-null  int64 
 1   produto_id   24527 non-null  int64 
 2   vendedor_id  24527 non-null  int64 
 3   data_compra  24527 non-null  object
 4   total        24527 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 958.2+ KB


In [21]:
# verificando tabela vendedores

consulta('SELECT * FROM VENDEDORES').head(5)

,vendedor_id,nome_vendedor
0,1,Ana Duarte
1,2,Daniel Siqueira
2,3,Nadia Oliveira
3,4,Millena Pereira
4,5,Paulo Calanca


Para entender o desempenho de vendas dos vendedores no ano de 2020, é preciso trabalhar com os dados que estão presentes apenas no ano de 2020.

In [22]:
# vamos filtrar apenas de 2020 com a soma das vendas dos vendedores

query = ''' SELECT PEDIDOS.VENDEDOR_ID, VENDEDORES.NOME_VENDEDOR, COUNT(PEDIDOS.TOTAL) AS quantidade_vendas
FROM PEDIDOS, VENDEDORES
WHERE strftime('%Y', PEDIDOS.DATA_COMPRA) = '2020' AND PEDIDOS.VENDEDOR_ID = VENDEDORES.VENDEDOR_ID
GROUP BY VENDEDORES.NOME_VENDEDOR
ORDER BY quantidade_vendas DESC '''

dados_vendedores = consulta(query)
dados_vendedores

,vendedor_id,nome_vendedor,quantidade_vendas
0,1,Ana Duarte,3427
1,2,Daniel Siqueira,3338
2,3,Nadia Oliveira,2782
3,4,Millena Pereira,2510
4,5,Paulo Calanca,2488


In [23]:
# vamos agora entender quais desses vendedores teve a melhor média de vendas

query = ''' SELECT PEDIDOS.VENDEDOR_ID, VENDEDORES.NOME_VENDEDOR, AVG(PEDIDOS.TOTAL) 'media das vendas'
FROM PEDIDOS, VENDEDORES
WHERE strftime('%Y', PEDIDOS.DATA_COMPRA) = '2020' AND PEDIDOS.VENDEDOR_ID = VENDEDORES.VENDEDOR_ID
GROUP BY VENDEDORES.NOME_VENDEDOR
ORDER BY AVG(PEDIDOS.TOTAL) DESC '''

dados_vendedores = consulta(query)
dados_vendedores

,vendedor_id,nome_vendedor,media das vendas
0,2,Daniel Siqueira,1972.338526
1,4,Millena Pereira,1894.614343
2,1,Ana Duarte,1843.452582
3,3,Nadia Oliveira,1818.342200
4,5,Paulo Calanca,1813.469855


# 4. Avançando nas relações

A Meteora agora deseja incrementar as suas vendas no **estado de São Paulo (SP)**, por acreditar ser uma região mais propícia às vendas já que tem uma boa concentração de pessoas e mercado também.

Mas será que São Paulo já não rende muitas vendas em relação a outros estados?

In [24]:
consulta('SELECT * FROM ITENS_PEDIDOS').head(10)

,id_nf,produto_id,pedido_id,quantidade,valor_unitario,valor_total,Estado,frete
0,1,41518,341,3,260,780,BR-BA,156.0
1,2,4307,1174,5,6175,30875,BR-RJ,6175.0
2,3,22407,1399,3,200,600,BR-PB,120.0
3,4,3820,1652,6,139,834,BR-DF,166.8
4,5,29012,2470,3,525,1575,BR-BA,315.0
5,6,72621,2980,3,179,537,BR-MG,107.4
6,7,86243,3443,4,350,1400,BR-PR,280.0
7,8,29012,3557,5,525,2625,BR-MS,525.0
8,9,61071,3585,3,450,1350,BR-SP,270.0
9,10,100961,3687,2,480,960,BR-GO,192.0


In [25]:
# vamos puxar a quantidade de pedidos que tivemos por estado

query = '''SELECT ITENS_PEDIDOS.ESTADO, COUNT(ITENS_PEDIDOS.PEDIDO_ID) AS 'quantidade de pedidos'
FROM ITENS_PEDIDOS
GROUP BY ITENS_PEDIDOS.ESTADO
ORDER BY COUNT(ITENS_PEDIDOS.PEDIDO_ID) DESC'''

dados_estado = consulta(query)
dados_estado

,Estado,quantidade de pedidos
0,BR-MA,974
1,BR-DF,953
2,BR-MT,937
3,BR-GO,933
4,BR-PA,932
5,BR-AL,928
6,BR-PE,927
7,BR-RR,925
8,BR-RO,925
9,BR-RN,921


Uma das ações que a Meteora pretende seguir para as vendas em SP é escolher dois de seus vendedores para que eles **foquem suas vendas** lá. Para essa escolha é interessante ter os profissionais que **mais tenham vendido** para as pessoas da região.

In [26]:
# vamos puxar agora os vendedores com mais de pedidos no Estado SP utilizando um metodo Join

query = '''SELECT VENDEDORES.NOME_VENDEDOR, COUNT(PEDIDOS.TOTAL) AS quantidade_vendas
FROM PEDIDOS
JOIN ITENS_PEDIDOS ON ITENS_PEDIDOS.PEDIDO_ID = PEDIDOS.PEDIDO_ID
JOIN VENDEDORES ON VENDEDORES.VENDEDOR_ID = PEDIDOS.VENDEDOR_ID
WHERE ITENS_PEDIDOS.ESTADO = 'BR-SP' AND PEDIDOS.VENDEDOR_ID = VENDEDORES.VENDEDOR_ID
GROUP BY VENDEDORES.NOME_VENDEDOR
ORDER BY quantidade_vendas DESC'''

dados_vendedores = consulta(query)
dados_vendedores

,nome_vendedor,quantidade_vendas
0,Daniel Siqueira,190
1,Paulo Calanca,182
2,Nadia Oliveira,181
3,Ana Duarte,180
4,Millena Pereira,157


# 5. Identificando Melhores **Resultados**

Em nosso estudo, queremos saber quais foram os produtos mais vendidos em 2019, assim teremos de como estavam nossos melhores produtos nessa epoca

In [27]:
# vamos puxar o top 10 produtos mais vendidos de 2019

query = '''SELECT PRODUTOS.PRODUTO, COUNT(PEDIDOS.PRODUTO_ID) AS quantidade_vendida
FROM PEDIDOS
JOIN PRODUTOS ON PRODUTOS.PRODUTO_ID = PEDIDOS.PRODUTO_ID
WHERE strftime('%Y', PEDIDOS.DATA_COMPRA) = '2019' AND PEDIDOS.PRODUTO_ID = PRODUTOS.PRODUTO_ID
GROUP BY PRODUTOS.PRODUTO
ORDER BY quantidade_vendida DESC
LIMIT 10'''

dados_produtos = consulta(query)
dados_produtos

,produto,quantidade_vendida
0,Vestido Recortes Cores,85
1,Calca Jeans Costuras,81
2,Saia Pregas Preta,73
3,Calca Alfaiataria Preta,66
4,Camisa Xadrez Azul,60
5,Bomber Tricot Off,56
6,Shorts Bordado Branco,55
7,Blusa Manga Estampa,54
8,Saia Midi Cinto,53
9,Saia Evase Cinza,53


In [28]:
# visualização produtos mais vendidos 2019

fig = px.bar(dados_produtos, x='quantidade_vendida', y='produto', color='produto', title='Produtos Mais Vendidos em 2019')
fig.update_layout(xaxis_title='Quantidade Vendida', yaxis_title='')
fig.show()

Agora queremos entender como foi nossas vendas no periodo de 2020, até onde. Aonde tivemos a maior alta e a menor baixa?

In [29]:
# primeiro vamos puxar todas as vendas separadas pelos meses

query = '''SELECT strftime('%m', PEDIDOS.DATA_COMPRA) AS mes, SUM(PEDIDOS.TOTAL) AS receita
FROM PEDIDOS
WHERE strftime('%Y', PEDIDOS.DATA_COMPRA) = '2020'
GROUP BY mes
ORDER BY mes ASC'''

dados_mes = consulta(query)
dados_mes

,mes,receita
0,01,2458252
1,02,2168386
2,03,2472297
3,04,2189559
4,05,2168099
5,06,1577791
6,07,1881010
7,08,2268550
8,09,2428227
9,10,2079915


In [30]:
# criando visualização dos periodo de 2020

fig = px.line(dados_mes, x='mes', y='receita', title='Vendas por Mês em 2020')
fig.update_layout(xaxis_title='Mês', yaxis_title='Receita')
fig.show()

# 6. Insigths para ação de Natal

A intenção da Meteora é iniciar sua ação no mês de dezembro em São Paulo, aproveitando as compras de natal para divulgar os produtos e lançar promoções para pedidos feitos nesse estado. Pensando nisso, vamos fornecer duas informações que podem contribuir na ação de dezembro.

A primeira informação é listar as marcas vendidas em São Paulo por quantidade de pedidos

In [31]:
# vamos puxar as marcas mais vendidas no estado de São Paulo por quantidade

query = '''SELECT PRODUTOS.MARCA, COUNT(ITENS_PEDIDOS.PEDIDO_ID) AS quantidade_vendida
FROM ITENS_PEDIDOS
JOIN PRODUTOS ON PRODUTOS.PRODUTO_ID = ITENS_PEDIDOS.PRODUTO_ID
WHERE ITENS_PEDIDOS.ESTADO = 'BR-SP' AND ITENS_PEDIDOS.PRODUTO_ID = PRODUTOS.PRODUTO_ID
GROUP BY PRODUTOS.MARCA
ORDER BY quantidade_vendida DESC'''

dados_marcas = consulta(query)
dados_marcas

,marca,quantidade_vendida
0,Zara,100
1,Mixed,58
2,Animale,44
3,Le Lis Blanc,26
4,Banana Republic,18
...,...,...
111,Topshop,1
112,Morena Rosa,1
113,Erre Erre,1
114,Bobô,1


A segunda informação é publicar os produtos que são mais vendidos na época de Natal no Brasil todo.

In [36]:
# puxando os produtos mais vendidos no mês de dezembro

query = '''SELECT PRODUTOS.PRODUTO, COUNT(ITENS_PEDIDOS.PEDIDO_ID) AS quantidade_vendida
FROM ITENS_PEDIDOS
JOIN PRODUTOS ON PRODUTOS.PRODUTO_ID = ITENS_PEDIDOS.PRODUTO_ID
JOIN PEDIDOS ON PEDIDOS.PEDIDO_ID = ITENS_PEDIDOS.PEDIDO_ID
WHERE strftime('%m', PEDIDOS.DATA_COMPRA) = '12'
GROUP BY PRODUTOS.PRODUTO
ORDER BY quantidade_vendida DESC'''

dados_produtos = consulta(query)
dados_produtos

,produto,quantidade_vendida
0,Vestido Recortes Cores,30
1,Vestido Preto Franzido,26
2,Calca Jeans Costuras,26
3,Calca Alfaiataria Preta,26
4,Camisa Xadrez Azul,25
...,...,...
196,Sneaker Monograma Bege,6
197,Casaqueto Estampa Geometrica,6
198,Regata Renda Branca,5
199,Body Estampa Coracoes,5
